# Data cleaning

In [528]:
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
import numpy as np
import statsmodels.tools as sm
import matplotlib.pyplot as plt
from linearmodels.iv import IV2SLS
import statsmodels.formula.api as smf
pd.options.mode.chained_assignment = None 

In [529]:
df = pd.read_csv('asia_20_women_main.csv',skipfooter=5)
df_leave = pd.read_csv('asia_20_women_leave.csv',skipfooter=5)
df_extra = pd.read_csv('asia_20_women_extra.csv',skipfooter=5)
df_parl = pd.read_csv('asia_20_women_parliament.csv', skipfooter=5)
df_ratio = pd.read_csv('asia_20_women_prop.csv', skipfooter=5)
df_fert = pd.read_csv('asia_20_women_fertility.csv', skipfooter=5)

C:\Users\aloi\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [530]:
year = pd.Categorical(df['Time'])

In [531]:
df.set_index(['Country Name','Time'], inplace=True)
df['Time'] = year

df_leave.set_index(['Country Name','Time'], inplace=True)
df_leave['Time'] = year

df_extra.set_index(['Country Name','Time'], inplace=True)
df_extra['Time'] = year

df_parl.set_index(['Country Name','Time'], inplace=True)
df_parl['Time'] = year

df_ratio.set_index(['Country Name','Time'], inplace=True)
df_ratio['Time'] = year

df_fert.set_index(['Country Name','Time'], inplace=True)
df_fert['Time'] = year

In [532]:
df_merged = pd.concat([df,df_leave,df_extra], axis=1)

In [533]:
df_merged

Country Code Time Code  \
Country Name Time                          
Bangladesh   2001          BGD    YR2001   
             2002          BGD    YR2002   
             2003          BGD    YR2003   
             2004          BGD    YR2004   
             2005          BGD    YR2005   
...                        ...       ...   
Vietnam      2016          VNM    YR2016   
             2017          VNM    YR2017   
             2018          VNM    YR2018   
             2019          VNM    YR2019   
             2020          VNM    YR2020   

                  Labor force participation rate, female (% of female population ages 15+) (national estimate) [SL.TLF.CACT.FE.NE.ZS]  \
Country Name Time                                                                                                                       
Bangladesh   2001                                                 ..                                                                    
             2002                                                 ..                                                                    
             2003                                   26.0699996948242                                                                    
             2004                                                 ..                                                                    
             2005                                                 ..                                                                    
...                                                              ...                                                                    
Vietnam      2016                                   71.9499969482422                                                                    
             2017                                   71.4400024414063                                                                    
             2018                                   70.4499969482422                                                                    
             2019                                   63.7999992370605                                                                    
             2020                                    62.189998626709                                                                    

                   Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate) [SL.TLF.CACT.FE.ZS]  \
Country Name Time                                                                                                                        
Bangladesh   2001                                          28.466000                                                                     
             2002                                          28.582001                                                                     
             2003                                          28.688999                                                                     
             2004                                          28.785000                                                                     
             2005                                          28.875999                                                                     
...                                                              ...                                                                     
Vietnam      2016                                          71.900002                                                                     
             2017                                          71.410004                                                                     
             2018                                          70.415001                                                                     
             2019                                          70.332001                                                                     
       

In [534]:
df_merged.replace('..', np.NaN, inplace=True)

In [535]:
df_merged.drop(['Country Code', 'Time Code', 'Time'], axis=1, inplace=True)

In [536]:
df_merged

Labor force participation rate, female (% of female population ages 15+) (national estimate) [SL.TLF.CACT.FE.NE.ZS]  \
Country Name Time                                                                                                                       
Bangladesh   2001                                                NaN                                                                    
             2002                                                NaN                                                                    
             2003                                   26.0699996948242                                                                    
             2004                                                NaN                                                                    
             2005                                                NaN                                                                    
...                                                              ...                                                                    
Vietnam      2016                                   71.9499969482422                                                                    
             2017                                   71.4400024414063                                                                    
             2018                                   70.4499969482422                                                                    
             2019                                   63.7999992370605                                                                    
             2020                                    62.189998626709                                                                    

                   Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate) [SL.TLF.CACT.FE.ZS]  \
Country Name Time                                                                                                                        
Bangladesh   2001                                          28.466000                                                                     
             2002                                          28.582001                                                                     
             2003                                          28.688999                                                                     
             2004                                          28.785000                                                                     
             2005                                          28.875999                                                                     
...                                                              ...                                                                     
Vietnam      2016                                          71.900002                                                                     
             2017                                          71.410004                                                                     
             2018                                          70.415001                                                                     
             2019                                          70.332001                                                                     
             2020                                          69.664001                                                                     

                  Adjusted savings: education expenditure (% of GNI) [NY.ADJ.AEDU.GN.ZS]  \
Country Name Time                                                                          
Bangladesh   2001                                                1.3                       
             2002                                               1.25                       
             2003                                               1.68     

In [537]:
cols = [0,6,7,10,11,13,14,15,16]
df_merged.drop(df_merged.columns[cols],axis=1,inplace=True)

In [538]:
# Number of na values for each independent variable 
def custom_key(na_values):
    return na_values[1]  # second parameter denotes the proportion of na value for the variable
na_lst = []
length = df_merged.shape[0]
for column in df_merged.columns:
    na_lst.append([column,df_merged[column].isna().sum()/length])
#na_lst.sort(key = custom_key, reverse = True)
na_lst

[['Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate) [SL.TLF.CACT.FE.ZS]',
  0.0],
 ['Adjusted savings: education expenditure (% of GNI) [NY.ADJ.AEDU.GN.ZS]',
  0.05],
 ['GDP per capita (constant 2015 US$) [NY.GDP.PCAP.KD]', 0.0],
 ['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]', 0.0],
 ['GDP per capita, PPP (constant 2017 international $) [NY.GDP.PCAP.PP.KD]',
  0.0],
 ['Paid leave of at least 14 weeks available to mothers (1=yes; 0=no) [SH.MMR.LEVE.AL]',
  0.0],
 ['Adolescent fertility rate (births per 1,000 women ages 15-19) [SP.ADO.TFRT]',
  0.05],
 ['Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]', 0.0]]

In [539]:
df1 = df_merged.dropna(axis=0)

In [540]:
df1.columns.to_list()

['Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate) [SL.TLF.CACT.FE.ZS]',
 'Adjusted savings: education expenditure (% of GNI) [NY.ADJ.AEDU.GN.ZS]',
 'GDP per capita (constant 2015 US$) [NY.GDP.PCAP.KD]',
 'GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]',
 'GDP per capita, PPP (constant 2017 international $) [NY.GDP.PCAP.PP.KD]',
 'Paid leave of at least 14 weeks available to mothers (1=yes; 0=no) [SH.MMR.LEVE.AL]',
 'Adolescent fertility rate (births per 1,000 women ages 15-19) [SP.ADO.TFRT]',
 'Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]']

In [541]:
df1.columns = ['female_labour','edu_exp','gdp_capita','gdp_growth','gdp_capita_ppp','paid_leave','adol_fertility','infant_mortality']

In [542]:
df1

female_labour           edu_exp   gdp_capita  gdp_growth  \
Country Name Time                                                             
Bangladesh   2001      28.466000               1.3   674.166973    5.077288   
             2002      28.582001              1.25   687.383308    3.833124   
             2003      28.688999              1.68   707.605094    4.739567   
             2004      28.785000              1.48   732.748841    5.239533   
             2005      28.875999             1.545   769.135980    6.535945   
...                          ...               ...          ...         ...   
Vietnam      2015      72.432999  4.61226021476106  2085.101349    6.679289   
             2016      71.900002  4.61226021476106  2191.819699    6.210812   
             2017      71.410004  4.61226021476106  2317.369069    6.812246   
             2018      70.415001  4.61226021476106  2456.791224    7.075789   
             2019      70.332001  4.61226021476106  2604.224166    7.017435   

                   gdp_capita_ppp  paid_leave adol_fertility  infant_mortality  
Country Name Time                                                               
Bangladesh   2001     1998.066320           0       109.1354              60.0  
             2002     2037.236310           0        106.017              57.0  
             2003     2097.168748           0       103.6338              54.3  
             2004     2171.688675           0       101.2506              51.8  
             2005     2279.531274           0        98.8674              49.4  
...                           ...         ...            ...               ...  
Vietnam      2015     6438.259854           1        32.9818              17.4  
             2016     6767.778832           1        31.9554              17.3  
             2017     7155.443187           1         30.929              17.1  
             2018     7585.943155           1        29.1518              17.0  
             2019     8041.178384           1        27.3746              16.9  

[418 rows x 8 columns]

In [543]:
cols = df1.columns[df1.dtypes.eq('object')]
df1[cols] = df1[cols].apply(pd.to_numeric, errors='coerce')

In [544]:
df1['ln_gdp_capita'] = np.log(df1['gdp_capita'])

In [545]:
df1.describe()

,female_labour,edu_exp,gdp_capita,gdp_growth,gdp_capita_ppp,paid_leave,adol_fertility,infant_mortality,ln_gdp_capita
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,53.289311,2.952893,8741.248113,5.529220,15999.606431,0.322967,35.606449,27.250478,8.146868
std,16.154938,1.319602,13286.354579,3.764202,20635.395055,0.468170,25.504537,20.736910,1.302135
min,16.167000,0.600000,344.880449,-13.129053,1055.653371,0.000000,1.261800,1.800000,5.843198
25%,45.068001,1.810330,1246.420837,3.511563,3739.503691,0.000000,13.156000,8.325000,7.128031
50%,53.730000,2.892234,2444.899874,5.794024,6958.932838,0.000000,31.828400,23.850000,7.801748
75%,63.681000,3.966897,8059.558671,7.306821,16774.425788,1.000000,51.111000,41.775000,8.994609
max,82.492996,7.030716,61173.904769,26.111494,98411.582062,1.000000,109.135400,82.600000,11.021476


In [546]:
df1.corr()

,female_labour,edu_exp,gdp_capita,gdp_growth,gdp_capita_ppp,paid_leave,adol_fertility,infant_mortality,ln_gdp_capita
female_labour,1.000000,0.041490,0.008654,0.097499,0.013509,0.060885,0.156919,-0.045335,-0.074040
edu_exp,0.041490,1.000000,0.128613,-0.155527,0.169017,0.095298,-0.350892,-0.437955,0.336891
gdp_capita,0.008654,0.128613,1.000000,-0.332853,0.969591,0.291660,-0.578848,-0.599237,0.889392
gdp_growth,0.097499,-0.155527,-0.332853,1.000000,-0.328409,0.004231,0.104835,0.222599,-0.341327
gdp_capita_ppp,0.013509,0.169017,0.969591,-0.328409,1.000000,0.182275,-0.555261,-0.604726,0.879604
paid_leave,0.060885,0.095298,0.291660,0.004231,0.182275,1.000000,-0.298935,-0.321389,0.281651
adol_fertility,0.156919,-0.350892,-0.578848,0.104835,-0.555261,-0.298935,1.000000,0.654427,-0.730856
infant_mortality,-0.045335,-0.437955,-0.599237,0.222599,-0.604726,-0.321389,0.654427,1.000000,-0.807625
ln_gdp_capita,-0.074040,0.336891,0.889392,-0.341327,0.879604,0.281651,-0.730856,-0.807625,1.000000


In [547]:
df1.index.levels[0]

Index(['Bangladesh', 'Bhutan', 'Brunei Darussalam', 'Cambodia', 'China',
       'India', 'Indonesia', 'Japan', 'Korea, Rep.', 'Lao PDR', 'Malaysia',
       'Maldives', 'Mongolia', 'Myanmar', 'Nepal', 'Pakistan', 'Philippines',
       'Singapore', 'Sri Lanka', 'Thailand', 'Timor-Leste', 'Vietnam'],
      dtype='object', name='Country Name')

# Hypothesis 1

### Model 1: Only IV, no Time and Entity Effects

In [548]:
formula1 = 'gdp_capita ~ 1 + female_labour'

In [549]:
model1_random = RandomEffects.from_formula(formula1, data = df1,check_rank=True)
model1_random_fit = model1_random.fit(cov_type='robust')
print(model1_random_fit)

                        RandomEffects Estimation Summary                        
Dep. Variable:             gdp_capita   R-squared:                        0.0626
Estimator:              RandomEffects   R-squared (Between):             -0.0818
No. Observations:                 418   R-squared (Within):               0.0694
Date:                Sun, Apr 24 2022   R-squared (Overall):             -0.0775
Time:                        15:01:42   Log-likelihood                   -3812.0
Cov. Estimator:                Robust                                           
                                        F-statistic:                      27.759
Entities:                          22   P-value                           0.0000
Avg Obs:                       19.000   Distribution:                   F(1,416)
Min Obs:                       19.000                                           
Max Obs:                       19.000   F-statistic (robust):             10.306
                            

### Model 2: IV + all controls, no Time and Entity Effects

In [550]:
formula2 = 'gdp_capita ~ 1 + female_labour + edu_exp + infant_mortality + adol_fertility + paid_leave'

In [551]:
model2_random = RandomEffects.from_formula(formula2, data = df1,check_rank=True)
model2_random_fit = model2_random.fit(cov_type='robust')
print(model2_random_fit)

                        RandomEffects Estimation Summary                        
Dep. Variable:             gdp_capita   R-squared:                        0.2393
Estimator:              RandomEffects   R-squared (Between):             -0.0168
No. Observations:                 418   R-squared (Within):               0.2555
Date:                Sun, Apr 24 2022   R-squared (Overall):             -0.0091
Time:                        15:01:43   Log-likelihood                   -3771.5
Cov. Estimator:                Robust                                           
                                        F-statistic:                      25.922
Entities:                          22   P-value                           0.0000
Avg Obs:                       19.000   Distribution:                   F(5,412)
Min Obs:                       19.000                                           
Max Obs:                       19.000   F-statistic (robust):             14.944
                            

### Model 3: IV + all controls, Time and Entity effects

In [552]:
formula3 = 'gdp_capita ~ 1 + female_labour + edu_exp + infant_mortality + adol_fertility + paid_leave + EntityEffects + TimeEffects'

In [553]:
model3 = PanelOLS.from_formula(formula3, data = df1, drop_absorbed=True, check_rank=True)
model3_fit = model3.fit()
print(model3_fit)

                          PanelOLS Estimation Summary                           
Dep. Variable:             gdp_capita   R-squared:                        0.2542
Estimator:                   PanelOLS   R-squared (Between):             -0.3081
No. Observations:                 418   R-squared (Within):              -0.1035
Date:                Sun, Apr 24 2022   R-squared (Overall):             -0.3023
Time:                        15:01:43   Log-likelihood                   -3695.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      25.432
Entities:                          22   P-value                           0.0000
Avg Obs:                       19.000   Distribution:                   F(5,373)
Min Obs:                       19.000                                           
Max Obs:                       19.000   F-statistic (robust):             25.432
                            

In [554]:
import numpy.linalg as la
from scipy import stats
def hausman(fe, re):
    b = fe.params
    B = re.params
    v_b = fe.cov
    v_B = re.cov
    df = b[np.abs(b) < 1e8].size
    chi2 = np.dot((b-B).T, la.inv(v_b-v_B).dot(b-B)) 
 
    pval = stats.chi2.sf(chi2, df)
    return chi2, df, pval

hausman_results = hausman(model3_fit, model2_random_fit) 
print('chi-Squared: ' + str(hausman_results[0]))
print('degrees of freedom: ' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

chi-Squared: 106.23890210477252
degrees of freedom: 6
p-Value: 1.248362025248711e-20


# Hypothesis 2

### Model 4: edu_exp\*female_labour

In [555]:
formula4 = 'gdp_capita ~ 1 + female_labour + edu_exp + infant_mortality + adol_fertility + paid_leave + edu_exp*female_labour +  EntityEffects + TimeEffects'

In [556]:
model4 = PanelOLS.from_formula(formula4, data = df1, drop_absorbed=True, check_rank=True)
model4_fit = model4.fit(cov_type='robust')
print(model4_fit)

                          PanelOLS Estimation Summary                           
Dep. Variable:             gdp_capita   R-squared:                        0.2611
Estimator:                   PanelOLS   R-squared (Between):             -0.2975
No. Observations:                 418   R-squared (Within):              -0.1143
Date:                Sun, Apr 24 2022   R-squared (Overall):             -0.2923
Time:                        15:01:46   Log-likelihood                   -3693.6
Cov. Estimator:                Robust                                           
                                        F-statistic:                      21.907
Entities:                          22   P-value                           0.0000
Avg Obs:                       19.000   Distribution:                   F(6,372)
Min Obs:                       19.000                                           
Max Obs:                       19.000   F-statistic (robust):             9.9290
                            

# Hypothesis 3

### Model 5: infant_mortality\*female_labour

In [557]:
formula5 = 'gdp_capita ~ 1 + female_labour + edu_exp + infant_mortality + adol_fertility + paid_leave + infant_mortality*female_labour + TimeEffects + EntityEffects'

In [558]:
model5 = PanelOLS.from_formula(formula5, data = df1, drop_absorbed=True, check_rank=True)
model5_fit = model5.fit(cov_type='robust')
print(model5_fit)

                          PanelOLS Estimation Summary                           
Dep. Variable:             gdp_capita   R-squared:                        0.2923
Estimator:                   PanelOLS   R-squared (Between):             -0.3177
No. Observations:                 418   R-squared (Within):              -0.1011
Date:                Sun, Apr 24 2022   R-squared (Overall):             -0.3116
Time:                        15:01:52   Log-likelihood                   -3684.6
Cov. Estimator:                Robust                                           
                                        F-statistic:                      25.605
Entities:                          22   P-value                           0.0000
Avg Obs:                       19.000   Distribution:                   F(6,372)
Min Obs:                       19.000                                           
Max Obs:                       19.000   F-statistic (robust):             8.8069
                            

# Hypothesis 4

### Model 6: fin_crisis\*female_labour

In [571]:
df1['fin_crisis'] = df1.index.map(lambda x: 1 if 2008<=x[1] else 0)
#df1['fin_crisis'] = df1.index.map(lambda x: 1 if 2008<=x[1] else 0)
#ok to omit time fixed effects

In [572]:
formula6 = 'gdp_capita ~ 1 + female_labour + edu_exp + infant_mortality + adol_fertility + paid_leave + fin_crisis + fin_crisis*female_labour + EntityEffects'

In [573]:
model6 = PanelOLS.from_formula(formula6, data = df1, drop_absorbed=True, check_rank=True)
model6_fit = model6.fit(cov_type='robust')
print(model6_fit)

                          PanelOLS Estimation Summary                           
Dep. Variable:             gdp_capita   R-squared:                        0.3297
Estimator:                   PanelOLS   R-squared (Between):             -0.1326
No. Observations:                 418   R-squared (Within):               0.3297
Date:                Sun, Apr 24 2022   R-squared (Overall):             -0.1195
Time:                        15:02:14   Log-likelihood                   -3732.3
Cov. Estimator:                Robust                                           
                                        F-statistic:                      27.334
Entities:                          22   P-value                           0.0000
Avg Obs:                       19.000   Distribution:                   F(7,389)
Min Obs:                       19.000                                           
Max Obs:                       19.000   F-statistic (robust):             17.115
                            

# Hypothesis 5

### Model 7: paid_leave * female_labour

In [574]:
formula7 = 'gdp_capita ~ 1 + female_labour + edu_exp + infant_mortality + adol_fertility + paid_leave + paid_leave*female_labour + TimeEffects + EntityEffects'

In [575]:
model7 = PanelOLS.from_formula(formula7, data = df1, drop_absorbed=True, check_rank=True)
model7_fit = model7.fit(cov_type='robust')
print(model7_fit)

                          PanelOLS Estimation Summary                           
Dep. Variable:             gdp_capita   R-squared:                        0.2776
Estimator:                   PanelOLS   R-squared (Between):             -0.2786
No. Observations:                 418   R-squared (Within):              -0.0593
Date:                Sun, Apr 24 2022   R-squared (Overall):             -0.2724
Time:                        15:02:20   Log-likelihood                   -3688.9
Cov. Estimator:                Robust                                           
                                        F-statistic:                      23.826
Entities:                          22   P-value                           0.0000
Avg Obs:                       19.000   Distribution:                   F(6,372)
Min Obs:                       19.000                                           
Max Obs:                       19.000   F-statistic (robust):             9.8478
                            

# Hypothesis 6 (2sls)

In [496]:
df2_merged = pd.concat([df,df_leave,df_extra, df_parl, df_fert], axis=1)
df2_merged.replace('..', np.NaN, inplace=True)
df2_merged.drop(['Country Code', 'Time Code', 'Time'], axis=1, inplace=True)
cols = [0,6,7,10,11,13,14,15,16]
df2_merged.drop(df2_merged.columns[cols],axis=1,inplace=True)

In [502]:
df2 = df2_merged.dropna(axis=0)

In [505]:
df2.columns

Index(['Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate) [SL.TLF.CACT.FE.ZS]',
       'Adjusted savings: education expenditure (% of GNI) [NY.ADJ.AEDU.GN.ZS]',
       'GDP per capita (constant 2015 US$) [NY.GDP.PCAP.KD]',
       'GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]',
       'GDP per capita, PPP (constant 2017 international $) [NY.GDP.PCAP.PP.KD]',
       'Paid leave of at least 14 weeks available to mothers (1=yes; 0=no) [SH.MMR.LEVE.AL]',
       'Adolescent fertility rate (births per 1,000 women ages 15-19) [SP.ADO.TFRT]',
       'Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]',
       'Proportion of seats held by women in national parliaments (%) [SG.GEN.PARL.ZS]',
       'Fertility rate, total (births per woman) [SP.DYN.TFRT.IN]'],
      dtype='object')

In [512]:
df2.columns = ['female_labour','edu_exp','gdp_capita','gdp_growth','gdp_capita_ppp','paid_leave','adol_fertility','infant_mortality','parl','tfr']

In [513]:
cols = df2.columns[df2.dtypes.eq('object')]
df2[cols] = df2[cols].apply(pd.to_numeric, errors='coerce')

In [514]:
tsls_formula = 'gdp_capita ~ 1 + [female_labour ~ parl +  tfr] + edu_exp + infant_mortality + adol_fertility + paid_leave' 

In [515]:
model_iv = IV2SLS.from_formula(formula = tsls_formula, data = df2)

In [516]:
results_iv = model_iv.fit(cov_type = 'unadjusted', debiased=True)

In [517]:
results_iv

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:             gdp_capita   R-squared:                      0.4283
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4209
No. Observations:                 388   F-statistic:                    61.001
Date:                Sun, Apr 24 2022   P-value (F-stat)                0.0000
Time:                        15:01:20   Distribution:                 F(5,382)
Cov. Estimator:            unadjusted                                         
                                                                              
                                Parameter Estimates                                 
====================================================================================
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
Intercept         1.729e+04     4009.2     4.3123     0.0000      9406.0   2.517e+04
adol_fertility      -215.72     33.328    -6.4727     0.0000     -281.25     -150.19
edu_exp             -1803.5     427.19    -4.2218     0.0000     -2643.4     -963.56
infant_mortality    -208.39     37.256    -5.5934     0.0000     -281.64     -135.13
paid_leave           2794.8     1171.0     2.3867     0.0175      492.45      5097.1
female_labour        161.24     78.602     2.0514     0.0409      6.6947      315.79
====================================================================================

Endogenous: female_labour
Instruments: parl, tfr
Unadjusted Covariance (Homoskedastic)
Debiased: True
IVResults, id: 0x1e60a47ffd0

In [518]:
results_iv.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 10.6409
P-value: 0.0012
Distributed: F(1,381)
WaldTestStatistic, id: 0x1e60c1d4d60

In [519]:
#insufficient evidence to reject H0, co2_damage is endogeneous 

In [520]:
first_stage_model = smf.ols(formula = 'female_labour ~ parl +  tfr', data = df2)

In [521]:
results_first_stage = first_stage_model.fit()

In [522]:
results_first_stage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          female_labour   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                     39.26
Date:                Sun, 24 Apr 2022   Prob (F-statistic):           3.05e-16
Time:                        15:01:27   Log-Likelihood:                -1600.9
No. Observations:                 388   AIC:                             3208.
Df Residuals:                     385   BIC:                             3220.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     47.2020      2.301     20.516      0.000      42.679      51.726
parl           0.8433      0.096      8.815      0.000       0.655       1.031
tfr           -3.0284      0.857     -3.533      0.000      -4.714      -1.343
==============================================================================
Omnibus:                        1.208   Durbin-Watson:                   0.163
Prob(Omnibus):                  0.547   Jarque-Bera (JB):                0.973
Skew:                          -0.076   Prob(JB):                        0.615
Kurtosis:                       3.193   Cond. No.                         56.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [523]:
df3 = df2.copy()

In [524]:
df3['resid'] = results_iv.resids

In [525]:
reg_aux = smf.ols(formula = 'resid ~ parl +  tfr + edu_exp + infant_mortality + adol_fertility + paid_leave', data = df3)

In [526]:
results_aux = reg_aux.fit()

In [527]:
r2 = results_aux.rsquared
n = results_aux.nobs
teststat = n*r2
pval = 1 - stats.chi2.cdf(teststat,1)

print(f'pval: {pval}')

pval: 0.006705450692517934


In [92]:
# high f-statistic, strong instrument
#we assume instrument exogeneity as there is one instrumental variable
#logic for instrument exogeneity assumption: z

In [131]:
#questions to ask at tutorial

#how to do hausman test for panel regression? prof say not necessary bc
#we shoudl always use fixed effects, but why can we just assume fixed effects assumptions
#is true?? what if its randomly distributed?
#for module can assume dont need

#do we need to standardise and normalise independent variables?
#different vars have different range (eg 10k - 20k), other vars is (1-10), one coeff may be very small or large compared to rest

#how to handle panel data in 2sls?
#can demeaning or first differencing

#using categorical gets absorbed entirely, why/how to avoid? we found papers that managed
#to do indicator for certain year (crisis), why theirs not absorbed?
#cannot, must remove time fixed effects first

#if we create interaction term for one hypotheses, should we keep in model for future hypotheses?
#leave interaction term in 

#prof refer to random effects model and 'no entity effect and time effect' as same, but diff
#result in linearmodels?

#no reason to do pooledols, usually will just do random effects vs fixed


#documentation says PanelOLS with no entity effect and time effect is PooledOLS